In [1]:
pip install scanpy

  Obtaining dependency information for scanpy from https://files.pythonhosted.org/packages/d1/b4/c408bac643afd670d2009219dbbb2076290e23fb1986937825c9a0d839c5/scanpy-1.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for anndata>=0.8 from https://files.pythonhosted.org/packages/ca/ee/2c4ba5ddd3d5664c02e8b4faefc94c7689601a29af4735b1491eb44e12a6/anndata-0.11.3-py3-none-any.whl.metadata
  Obtaining dependency information for h5py>=3.6 from https://files.pythonhosted.org/packages/0d/74/1009b663387c025e8fa5f3ee3cf3cd0d99b1ad5c72eeb70e75366b1ce878/h5py-3.12.1-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for legacy-api-wrap>=1.4 from https://files.pythonhosted.org/packages/3c/1d/9324c70629dfe4395f9122ab331cf245d3cce6ded851aa8a0a8ae264c4e6/legacy_api_w

In [12]:
import os
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [13]:
project_path = "/Users/Swarnali_1/scrna_seq_scanpy/data/"

samples = {"not_injured" : "GSM8289594_QSC_GFP_filtered_feature_bc_matrix.h5",
"injured" : "GSM8289595_ASC_GFP_filtered_feature_bc_matrix.h5"}

adatas = {}

for sample_id, file_id in samples.items():
    path = os.path.join(project_path, file_id)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

for sample_id, sample in adatas.items():
    print(f"Sample Name: {sample_id}, Sample: {sample}")


adata = ad.concat(adatas, label = "sample")
adata.obs_names_make_unique()
print(adata.obs["sample"].value_counts())


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Sample Name: not_injured, Sample: AnnData object with n_obs × n_vars = 8449 × 32285
    var: 'gene_ids', 'feature_types', 'genome'
Sample Name: injured, Sample: AnnData object with n_obs × n_vars = 6429 × 32285
    var: 'gene_ids', 'feature_types', 'genome'
sample
not_injured    8449
injured        6429
Name: count, dtype: int64


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [18]:
# Quality control

## Mitochondrial content
adata.var["mt"] = adata.var_names.str.startswith("Mt-")
## Ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith("RPS", "RPL")
## Hemoglobin genes
adata.var["hb"] = adata.var_names.str.startswith("^HB[^(P)]")

## Calculate quality control metrics
sc.pp.calculate_qc_metrics(adata, qc_vars = ["mt", "ribo", "hb"])


(                    n_genes_by_counts  log1p_n_genes_by_counts  total_counts  \
 AAACCCAAGAGATTCA-1               3130                 8.049108        8428.0   
 AAACCCAAGGAATGTT-1               3049                 8.022897        7313.0   
 AAACCCACAAATTAGG-1               1345                 7.204893        2747.0   
 AAACCCACATACTGAC-1               1644                 7.405496        3358.0   
 AAACCCACATAGATGA-1               1429                 7.265430        2711.0   
 ...                               ...                      ...           ...   
 TTTGTTGGTTTCGCTC-1               2597                 7.862497        7032.0   
 TTTGTTGTCAAGGCTT-1               1933                 7.567346        4390.0   
 TTTGTTGTCCGATCGG-1               3108                 8.042056        9870.0   
 TTTGTTGTCCTATTTG-1               3072                 8.030410        8476.0   
 TTTGTTGTCGAATGCT-1               3630                 8.197263       11109.0   
 
                     log1p